# Populate Privacy Tab from Privacy-AI_refactored.csv

This notebook fills the Privacy (Priv/LPRP) sheet in the 2024-2025 Statistical Reports template from the consolidated Privacy dataset.

- Select institution, reporting period, and language.
- Generates a new .xlsx file with the Privacy tab populated.
- Uses Privacy_ForConsumption mapping for cell targets.
- Includes a mapping accuracy report based on dataset header similarity.


In [ ]:
# %pip install ipywidgets
import re
import unicodedata
from pathlib import Path
from difflib import SequenceMatcher

import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import column_index_from_string, get_column_letter
from openpyxl.worksheet.formula import ArrayFormula
from IPython.display import display


In [ ]:
CSV_URL = None  # TODO: set to hosted URL (e.g., https://.../Privacy-AI_refactored.csv)
CSV_FALLBACK_PATH = "Privacy-AI_refactored.csv"

TEMPLATE_EN = "2024-2025 Statistical Reports.xlsx"
TEMPLATE_FR = "2024-2025 Rapports statistiques.xlsx"
MAPPING_SHEET = "Privacy_ForConsumption"
PRIV_SHEET_EN = "Priv"
PRIV_SHEET_FR = "LPRP"
PRIV_DATE_START_CELL = "H8"
PRIV_DATE_END_CELL = "L8"

SCHEMA_PATH = "combined_privacy_form_data.xlsx"
DATASET_HEADER_URL = "https://open.canada.ca/data/dataset/236294e1-bc74-486f-ab97-422227bc8832/resource/7b019922-e25c-454e-addf-12c8d72952c0/download/privacy-dataset-2023-24-1.xlsx"


In [ ]:
DIRECT_REF_RE = re.compile(r"=\s*'?([^'!]+)'?!\$?([A-Z]+)\$?(\d+)")
ARRAY_REF_RE = re.compile(r"=TRANSPOSE\('?([^'!]+)'?!\$?([A-Z]+)\$?(\d+):\$?([A-Z]+)\$?(\d+)\)")
ID_RE = re.compile(r"Row(\d+)([a-z]?)Cell(\d+)([a-z]?)$", re.IGNORECASE)


def normalize_sub_key(value):
    if value is None or pd.isna(value):
        return None
    text = str(value).strip()
    if text == "" or text.lower() == "nan":
        return None
    if text.endswith(".0"):
        text = text[:-2]
    return text.replace(",", ".")


def suffix_rank(value):
    if not value:
        return 0
    return ord(value.lower()) - ord("a") + 1


def parse_id_sort_key(id_value):
    if not isinstance(id_value, str):
        id_value = str(id_value)
    match = ID_RE.search(id_value)
    if not match:
        return (10**9, 0, 10**9, 0, id_value)
    row_num = int(match.group(1))
    row_suffix = suffix_rank(match.group(2))
    cell_num = int(match.group(3))
    cell_suffix = suffix_rank(match.group(4))
    return (row_num, row_suffix, cell_num, cell_suffix, id_value)


def sanitize_filename(text):
    normalized = unicodedata.normalize("NFKD", str(text)).encode("ascii", "ignore").decode("ascii")
    cleaned = re.sub(r"[^A-Za-z0-9._-]+", "_", normalized).strip("_")
    return cleaned or "institution"


def expand_range(start_col, start_row, end_col, end_row):
    c1 = column_index_from_string(start_col)
    c2 = column_index_from_string(end_col)
    r1 = int(start_row)
    r2 = int(end_row)
    if c2 < c1:
        c1, c2 = c2, c1
    if r2 < r1:
        r1, r2 = r2, r1
    cells = []
    for row in range(r1, r2 + 1):
        for col in range(c1, c2 + 1):
            cells.append(f"{get_column_letter(col)}{row}")
    return cells


def parse_direct_ref(formula):
    match = DIRECT_REF_RE.match(str(formula).strip())
    if not match:
        return None, None
    sheet = match.group(1).strip("'")
    cell = f"{match.group(2)}{match.group(3)}"
    return sheet, cell


def parse_array_formula(text):
    match = ARRAY_REF_RE.match(str(text).strip())
    if not match:
        return None, []
    sheet = match.group(1).strip("'")
    cells = expand_range(match.group(2), match.group(3), match.group(4), match.group(5))
    return sheet, cells


def build_mapping_from_sheet(ws):
    row_to_target = {}
    seen_refs = set()

    for row in range(1, ws.max_row + 1):
        value = ws.cell(row=row, column=2).value
        if isinstance(value, ArrayFormula):
            if value.ref in seen_refs:
                continue
            seen_refs.add(value.ref)
            ref_match = re.match(r"B(\d+):B(\d+)", value.ref)
            if not ref_match:
                continue
            start_row = int(ref_match.group(1))
            end_row = int(ref_match.group(2))
            sheet, cells = parse_array_formula(value.text)
            if not sheet or not cells:
                continue
            for idx, target_row in enumerate(range(start_row, end_row + 1)):
                if idx < len(cells):
                    row_to_target[target_row] = (sheet, cells[idx])

    for row in range(1, ws.max_row + 1):
        if row in row_to_target:
            continue
        value = ws.cell(row=row, column=2).value
        if isinstance(value, str) and value.startswith("="):
            sheet, cell = parse_direct_ref(value)
            if sheet and cell:
                row_to_target[row] = (sheet, cell)

    inst_target = None
    sub_map = {}

    for row in range(1, ws.max_row + 1):
        key = ws.cell(row=row, column=1).value
        sub = ws.cell(row=row, column=3).value
        if key == "Inst":
            inst_target = row_to_target.get(row)
            continue
        if key and sub is not None:
            sub_key = normalize_sub_key(sub)
            if not sub_key:
                continue
            target = row_to_target.get(row)
            if not target:
                continue
            sub_map.setdefault(sub_key, []).append(target)

    return inst_target, sub_map


def is_formula_cell(ws, cell):
    target = ws[cell]
    if target.data_type == "f":
        return True
    value = target.value
    return isinstance(value, str) and value.startswith("=")


def load_privacy_data(csv_url=None, fallback_path=CSV_FALLBACK_PATH):
    df = None
    if csv_url:
        try:
            df = pd.read_csv(
                csv_url,
                dtype={"section_number": "string", "subsection_number": "string"},
                low_memory=False,
            )
        except Exception as exc:
            print(f"Failed to read CSV from URL ({exc}); falling back to {fallback_path}.")

    if df is None:
        df = pd.read_csv(
            fallback_path,
            dtype={"section_number": "string", "subsection_number": "string"},
            low_memory=False,
        )

    df["ReportingPeriodStart"] = pd.to_datetime(df["ReportingPeriodStart"], errors="coerce")
    df["ReportingPeriodEnd"] = pd.to_datetime(df["ReportingPeriodEnd"], errors="coerce")
    df["report_start"] = df["ReportingPeriodStart"].dt.date
    df["report_end"] = df["ReportingPeriodEnd"].dt.date

    df["gc_orgID"] = pd.to_numeric(df["gc_orgID"], errors="coerce").astype("Int64")

    sub = df["subsection_number"].fillna("").astype("string")
    sub = sub.where(~sub.str.lower().eq("nan"), "")
    sec = df["section_number"].fillna("").astype("string")
    sec = sec.where(~sec.str.lower().eq("nan"), "")

    sub_key = sub.mask(sub == "", sec)
    df["sub_key"] = sub_key.map(normalize_sub_key)

    return df


def build_output_filename(inst_name, start_date, lang):
    year = pd.to_datetime(start_date, errors="coerce").year
    year_label = f"{year}" if pd.notna(year) else "unknown"
    inst_slug = sanitize_filename(inst_name)[:60]
    lang_label = "EN" if str(lang).lower().startswith("en") else "FR"
    return f"Privacy_Report_{year_label}_{lang_label}_{inst_slug}.xlsx"


def populate_workbook(wb, subset, inst_name, start_date, end_date, lang):
    mapping_ws = wb[MAPPING_SHEET]
    inst_target, sub_map = build_mapping_from_sheet(mapping_ws)

    if inst_target:
        inst_sheet, inst_cell = inst_target
    else:
        inst_sheet = PRIV_SHEET_EN if str(lang).lower().startswith("en") else PRIV_SHEET_FR
        inst_cell = "H6"

    wb[inst_sheet][inst_cell] = inst_name

    if pd.notna(start_date):
        wb[inst_sheet][PRIV_DATE_START_CELL] = pd.to_datetime(start_date).date()
    if pd.notna(end_date):
        wb[inst_sheet][PRIV_DATE_END_CELL] = pd.to_datetime(end_date).date()

    warnings = []
    skipped_formula = 0
    skipped_examples = []

    for sub_key, targets in sub_map.items():
        sub_rows = subset[subset["sub_key"] == sub_key]
        if sub_rows.empty:
            warnings.append(f"Subsection {sub_key}: no matching rows in data.")
            continue
        sub_rows = sub_rows.copy()
        sub_rows["sort_key"] = sub_rows["id"].map(parse_id_sort_key)
        sub_rows = sub_rows.sort_values("sort_key")
        values = sub_rows["value"].tolist()

        if len(values) != len(targets):
            warnings.append(
                f"Subsection {sub_key}: template has {len(targets)} cells, data has {len(values)}; filled {min(len(values), len(targets))}."
            )

        for value, target in zip(values, targets):
            if pd.isna(value):
                continue
            sheet_name, cell = target
            if is_formula_cell(wb[sheet_name], cell):
                skipped_formula += 1
                if len(skipped_examples) < 10:
                    skipped_examples.append(f"{sheet_name}!{cell}")
                continue
            wb[sheet_name][cell] = value

    if skipped_formula:
        sample = ", ".join(skipped_examples)
        warnings.append(
            f"Skipped {skipped_formula} formula cells (auto-calculated), e.g. {sample}."
        )

    return warnings


def generate_privacy_report(df, gc_org_id, report_start, lang="En", output_path=None):
    report_start = pd.to_datetime(report_start, errors="coerce").date()

    template_path = TEMPLATE_EN if str(lang).lower().startswith("en") else TEMPLATE_FR
    if not Path(template_path).exists():
        raise FileNotFoundError(f"Template not found: {template_path}")

    subset = df[(df["gc_orgID"] == gc_org_id) & (df["report_start"] == report_start)]
    if subset.empty:
        raise ValueError("No rows found for the selected institution and period.")

    first_row = subset.iloc[0]
    inst_name = first_row["institution_fr"] if str(lang).lower().startswith("fr") else first_row["institution_en"]
    start_date = first_row["ReportingPeriodStart"]
    end_date = first_row["ReportingPeriodEnd"]

    wb = load_workbook(template_path, data_only=False)
    warnings = populate_workbook(wb, subset, inst_name, start_date, end_date, lang)

    if output_path is None:
        output_path = build_output_filename(inst_name, start_date, lang)

    wb.save(output_path)
    return output_path, warnings


In [ ]:
df = load_privacy_data(CSV_URL, CSV_FALLBACK_PATH)
print(f"Loaded {len(df):,} rows from Privacy-AI_refactored.csv")


In [ ]:
# Example (manual usage without widgets):
# output_path, warnings = generate_privacy_report(
#     df,
#     gc_org_id=1234,
#     report_start="2024-04-01",
#     lang="En",
# )
# print("Saved:", output_path)
# if warnings:
#     print("Warnings:")
#     for item in warnings[:20]:
#         print("-", item)


In [ ]:
try:
    import ipywidgets as widgets

    inst_df = (
        df[["gc_orgID", "institution_en", "institution_fr"]]
        .dropna(subset=["gc_orgID"])
        .drop_duplicates()
        .sort_values("gc_orgID")
    )
    inst_df["gc_orgID"] = inst_df["gc_orgID"].astype(int)

    inst_options = [
        (f"{row.gc_orgID} | {row.institution_en}", row.gc_orgID)
        for row in inst_df.itertuples(index=False)
    ]

    period_dates = sorted(df["report_start"].dropna().unique())
    period_options = [(f"{d.year}-{d.year + 1}", d) for d in period_dates]

    inst_dropdown = widgets.Dropdown(options=inst_options, description="Institution", layout=widgets.Layout(width="80%"))
    period_dropdown = widgets.Dropdown(options=period_options, description="Period")
    lang_dropdown = widgets.Dropdown(options=["En", "Fr"], description="Language")
    generate_btn = widgets.Button(description="Generate XLSX", button_style="primary")
    output_area = widgets.Output()

    def on_click(_):
        output_area.clear_output()
        with output_area:
            try:
                output_path, warnings = generate_privacy_report(
                    df,
                    gc_org_id=inst_dropdown.value,
                    report_start=period_dropdown.value,
                    lang=lang_dropdown.value,
                )
                print(f"Saved: {output_path}")
                if warnings:
                    print("Warnings:")
                    for item in warnings[:20]:
                        print("-", item)
            except Exception as exc:
                print("Error:", exc)

    generate_btn.on_click(on_click)
    display(widgets.VBox([inst_dropdown, period_dropdown, lang_dropdown, generate_btn, output_area]))
except ImportError:
    print("ipywidgets not installed. Run %pip install ipywidgets, or call generate_privacy_report(...) manually.")


## Mapping accuracy (dataset headers vs schema titles)

This cell builds a field-level accuracy estimate based on dataset header similarity to schema titles, and the presence of a target cell in the workbook mapping.


In [ ]:
def build_dataset_headers(url):
    raw = pd.read_excel(url, header=None)
    header_rows = raw.iloc[:3].copy()
    header_rows = header_rows.ffill(axis=1)
    headers = []
    for col in header_rows.columns:
        parts = []
        for row in range(3):
            val = header_rows.iat[row, col]
            if pd.isna(val):
                continue
            text = str(val).strip()
            if not text:
                continue
            if not parts or parts[-1] != text:
                parts.append(text)
        headers.append(' - '.join(parts) if parts else None)
    if headers and headers[0] is None:
        headers[0] = 'Institution'
    return headers


def similarity(a, b):
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, str(a).casefold(), str(b).casefold()).ratio()


schema = pd.read_excel(SCHEMA_PATH)
value_rows = schema[~schema['id'].isin(['NameOfInstitution', 'ReportingPeriodStart', 'ReportingPeriodEnd'])].copy()
value_rows = value_rows.reset_index(drop=True)
value_rows['schema_index'] = value_rows.index
value_rows['sub_key'] = value_rows['subsection_number'].apply(normalize_sub_key)
value_rows['sub_key'] = value_rows['sub_key'].where(value_rows['sub_key'].notna(), value_rows['section_number'].apply(normalize_sub_key))
value_rows['id_sort'] = value_rows['id'].map(parse_id_sort_key)

value_rows_sorted = value_rows.sort_values(['sub_key', 'id_sort', 'schema_index']).copy()
value_rows_sorted['sub_index'] = value_rows_sorted.groupby('sub_key').cumcount()

wb = load_workbook(TEMPLATE_EN, data_only=False)
map_ws = wb[MAPPING_SHEET]

# build mapping lookup
inst_target, sub_map = build_mapping_from_sheet(map_ws)
mapping_lookup = {}
for sub_key, targets in sub_map.items():
    for idx, target in enumerate(targets):
        mapping_lookup[(sub_key, idx)] = target

headers = build_dataset_headers(DATASET_HEADER_URL)

rows = []
for row in value_rows_sorted.itertuples(index=False):
    sub_key = row.sub_key
    sub_index = row.sub_index
    mapping = mapping_lookup.get((sub_key, sub_index))

    dataset_col_index = row.schema_index + 1
    dataset_col_name = headers[dataset_col_index] if dataset_col_index < len(headers) else None

    if mapping:
        target_sheet, target_cell = mapping
        target_is_formula = is_formula_cell(wb[target_sheet], target_cell)
    else:
        target_sheet = None
        target_cell = None
        target_is_formula = None

    title_en = row.title_en
    header_similarity = similarity(dataset_col_name, title_en)

    rows.append({
        'dataset_column_index': dataset_col_index,
        'dataset_column_name': dataset_col_name,
        'html_form_id': row.id,
        'section_number': row.section_number,
        'subsection_number': row.subsection_number,
        'sub_key': sub_key,
        'title_en': title_en,
        'xlsx_sheet': target_sheet,
        'xlsx_cell': target_cell,
        'xlsx_ref': f"{target_sheet}!{target_cell}" if target_sheet and target_cell else None,
        'xlsx_cell_is_formula': target_is_formula,
        'mapping_status': 'ok' if mapping else 'missing_xlsx_mapping',
        'header_similarity': header_similarity,
    })

accuracy_df = pd.DataFrame(rows)
accuracy_df['estimated_accuracy'] = accuracy_df['header_similarity']
accuracy_df['accuracy_band'] = pd.cut(
    accuracy_df['estimated_accuracy'],
    bins=[-0.01, 0.35, 0.6, 1.01],
    labels=['low', 'medium', 'high'],
)

accuracy_path = Path('privacy_rosetta_mapping_accuracy.csv')
accuracy_df.to_csv(accuracy_path, index=False)

mismatch_df = accuracy_df[(accuracy_df['mapping_status'] != 'ok') | (accuracy_df['header_similarity'] < 0.35)].copy()
mismatch_path = Path('privacy_rosetta_mismatches.csv')
mismatch_df.to_csv(mismatch_path, index=False)

print(f"Wrote {accuracy_path}")
print(f"Wrote {mismatch_path}")
print("Summary:")
print(accuracy_df['accuracy_band'].value_counts().to_string())
